In [ ]:
tcf(filename::String) = (@__DIR__) * "\\" * filename;

In [ ]:
import Pkg;
Pkg.activate("E:/gitcl/Lec2/ccmech_julia")

In [ ]:
Pkg.instantiate()

# Моделирование растяжения плоского образца

$\hspace{0.25cm}$ Все известных нам в природе материалы элементов конструкций деформируются под действием приложенных к ним сил. В расчетах материалы отличаются на упругие и пластичные.
Процесс деформирования в каждом элементе, приложенные силы, производят некоторую работу. Если материал упругий, то эта работа запасается в виде потенциальной энергии, которая освобождается по мере того, как уменьшаются деформирующие силы. Если материал пластичный, то работа, произведенная приложенными к нему внешними силами, пойдет на изменение его физического состояния или вызовет нагревание. Чем больше жесткость материала, тем меньше при данной системе приложенных сил запасенная работа.

$\hspace{0.25cm}$ В пределах малых деформаций существует линейная зависимость между физическими свойствами материала и напряжениями и деформациями. Эта зависимость носит название обобщенного закона Гука:
$$
\begin{cases}
   \varepsilon_x=\frac{1}{E}[\mathbf{\sigma}_x-\nu (\mathbf{\sigma}_y+\mathbf{\sigma}_z)]; \\
   \varepsilon_y=\frac{1}{E}[\mathbf{\sigma}_y-\nu (\mathbf{\sigma}_x+\mathbf{\sigma}_z)]; \\
   \varepsilon_z=\frac{1}{E}[\mathbf{\sigma}_z-\nu (\mathbf{\sigma}_x+\mathbf{\sigma}_y)]; \\
   \tau_{xy}=G\gamma_{xy}; \\
   \tau_{yz}=G\gamma_{yz}; \\
   \tau_{zx}=G\gamma_{zx}; \\
   \end{cases}
$$

где $\varepsilon$ - линейная деформация вдоль оси, [б.р.]; $\mathbf{\sigma}$ - нормальное напряжение, [$\frac{Н}{м^2}$ = Па]; $E$ - модуль Юнга, [Па];  $\nu$ - коэффициент Пуассона, [б.р.]; $\tau$ - касательное напряжение, [$\frac{Н}{м^2}$ = Па]; $\gamma_{xy}$ - угловая деформация, [б.р.]; $G$ - модуль сдвига, $G=\frac{E}{2(1+\nu)}$ [Па].

$\hspace{0.25cm}$ Коэффициент Пуассона показывает во сколько раз изменяется поперечное сечение деформированного тела при его растяжении или сжатии (величина отношения относительного поперечного сжатия к относительному продольному растяжению) и зависит от природы материала. Модуль Юнга (модуль упругости) - коэффициент, характеризующий сопротивление материала растяжению сжатию при упругой деформации. Два этих коэффициента полностью характеризуют упругие свойства изотропного материала.

Также закон гука можно записать в обратной форме:
$$
\begin{cases}
   \mathbf{\sigma}_x=\lambda tr(\varepsilon)+ 2\mu\mathbf{\varepsilon}_x; \\
   \mathbf{\sigma}_y=\lambda tr(\varepsilon)+ 2\mu\mathbf{\varepsilon}_y; \\
   \mathbf{\sigma}_z=\lambda tr(\varepsilon)+ 2\mu\mathbf{\varepsilon}_z; \\
   \tau_{xy}=\mu \gamma_{xy}; \\
   \tau_{yz}=\mu \gamma_{yz}; \\
   \tau_{zx}=\mu \gamma_{zx}; \\
   \end{cases}
$$
где $\lambda = \frac{\nu E}{(1+\nu)(1-2\nu)}, \mu=\frac{E}{2(1+\nu)}=G$ - коэффициенты Ляме.

$\hspace{0.25cm}$ Часто механические тела испытывают одновременно более одного типа нагрузки; это называется комбинированным напряжением. В наиболее общем случае, называемом трёхосным напряжением, напряжение отлично от нуля на каждом элементе поверхности. Комбинированные напряжения нельзя описать одним вектором. Поэтому даже если материал подвергается одинаковому напряжению во всем объёме тела, напряжение на любой воображаемой поверхности будет зависеть от ориентации этой поверхности. Для описания таких случаев закон Гука записывают в тензорной форме:
$$
    \mathbf{\sigma} = {}^{4}\mathbf{E} \cdot \mathbf{\varepsilon}
$$

где $\mathbf{\sigma}$ - тензор напряжений, $\mathbf{\varepsilon}$ - тензор деформаций, ${}^{4}\mathbf{E}$ - тензор упругости.
Или же, используя нотацию Фойгта, согласно которой тензора напряжений и деформаций представляются в виде векторов содержащих 6 компонент:
$$
    \mathbf{\sigma_v} = \left\{
        \begin{matrix}
            \sigma_{x}\\
            \sigma_{y}\\
            \sigma_{z}\\
            \tau_{yz}\\
            \tau_{xz}\\
            \tau_{xy}\\
        \end{matrix}
    \right\}
    \hspace{0.7cm}
    \&
    \hspace{0.7cm}
    \mathbf{\varepsilon_v} = \left\{
        \begin{matrix}
            \varepsilon_{x}\\
            \varepsilon_{y}\\
            \varepsilon_{z}\\
            \gamma_{yz}\\
            \gamma_{xz}\\
            \gamma_{xy}\\
        \end{matrix}
    \right\}

    
$$

Тогда тензор упругости примет вид
$$
    \mathbf{E} = \frac{E(1-\nu)}{(1+\nu)(1-2\nu)}\left[
        \begin{matrix}
        1 & \frac{\nu}{1-\nu} & \frac{\nu}{1-\nu} & 0 & 0 & 0 \\
        \frac{\nu}{1-\nu} & 1 & \frac{\nu}{1-\nu} & 0 & 0 & 0 \\
        \frac{\nu}{1-\nu} & \frac{\nu}{1-\nu} & 1 & 0 & 0 & 0 \\
        0 & 0 & 0 & \frac{1-2\nu}{2(1-\nu)} & 0 & 0 \\
        0 & 0 & 0 & 0 & \frac{1-2\nu}{2(1-\nu)} & 0 \\
        0 & 0 & 0 & 0 & 0 & \frac{1-2\nu}{2(1-\nu)} \\

        \end{matrix}
    \right]
$$


## Создание геометрии

![image](less2.jpg)

In [ ]:
# ГОСТ 1497-84, образец плоский, тип I, образец номер №6
mm = 1e-3;
B = 40mm;
l0 = 140mm;
a0 = 20mm;
b0 = 30mm;
l = l0+2*sqrt(a0*b0);
h1 = 80mm;
r = (B-b0)/2;

In [ ]:
using Gmsh
using GridapGmsh
using Gridap

gmsh.initialize()

gmsh.option.setNumber("General.Terminal", 0);
ms_max = 1;
ms_min = 0.1;
MSFC = 3;
EL_ORDER = 1
gmsh.option.setNumber("Mesh.MeshSizeFromCurvature", MSFC);
# gmsh.option.setNumber("Mesh.MeshSizeMax", diam/2);
# gmsh.option.setNumber("Mesh.MeshSizeMin", diam/5);
gmsh.option.setNumber("Mesh.MaxNumThreads3D", 8);
gmsh.option.setNumber("Mesh.ElementOrder", EL_ORDER);
# gmsh.option.setNumber("Mesh.Algorithm3D", 9);
# gmsh.option.setNumber("Mesh.SubdivisionAlgorithm", 2);

gmsh.model.add("L_2_1")

Lc1 = 1mm;

factory = gmsh.model.geo;
factory.addPoint(0,             0,      0, Lc1, 1);#left-bottom corner
factory.addPoint(h1,            0,      0, Lc1, 2); #begin of first arc
factory.addPoint(h1+r,          0,      0, Lc1, 3); #center of first arc
factory.addPoint(h1+r,          r,      0, Lc1, 4); #end of first arc
factory.addPoint(h1+r+l,        r,      0, Lc1, 5); #begin of second arc
factory.addPoint(h1+r+l,        0,      0, Lc1, 6); #center of second arc
factory.addPoint(h1+r+l+r,      0,      0, Lc1, 7); #begin of second arc
factory.addPoint(h1+r+l+r+h1,   0,      0, Lc1, 8); #right-buttom corner
factory.addPoint(h1+r+l+r+h1,   B,      0, Lc1, 9); #right-top corner
factory.addPoint(h1+r+l+r,      B,      0, Lc1, 10); #begin of third arc
factory.addPoint(h1+r+l,        B,      0, Lc1, 11); #center of third arc
factory.addPoint(h1+r+l,        B-r,    0, Lc1, 12); #end of third arc
factory.addPoint(h1+r,          B-r,    0, Lc1, 13); #begin of fourth arc
factory.addPoint(h1+r,          B,      0, Lc1, 14); #center of fourth arc
factory.addPoint(h1,            B,      0, Lc1, 15); #end of fourth arc
factory.addPoint(0,             B,      0, Lc1, 16); #end of fourth arc


factory.addLine(1, 2, 1);
factory.addCircleArc(2, 3, 4, 2);

factory.addLine(4, 5, 3);
factory.addCircleArc(5, 6, 7, 4);

factory.addLine(7, 8, 5);
factory.addLine(8, 9, 6);
factory.addLine(9, 10, 7);
factory.addCircleArc(10, 11, 12, 8);

factory.addLine(12, 13, 9);
factory.addCircleArc(13, 14, 15, 10);
factory.addLine(15, 16, 11);
factory.addLine(16, 1, 12);

factory.addPhysicalGroup(1, [12], -1,"left");
factory.addPhysicalGroup(1, [6], -1, "right");
factory.addPhysicalGroup(1, [1, 2, 3, 4, 5, 7, 8, 9, 10, 11], -1, "free");

factory.synchronize();

factory.addCurveLoop(collect(1:12), 13);



factory.addPlaneSurface([13], 6)


factory.synchronize()

factory.addPhysicalGroup(2, [6], -1, "test_sample");

factory.synchronize()

gmsh.model.mesh.generate(2)


name = "test_sample.msh" |> tcf;
print(name)
gmsh.write(name)

# gmsh.fltk.run()

gmsh.finalize()


In [ ]:

model = GmshDiscreteModel(name);

vtk_file = "model_speciment" |> tcf;
writevtk(model, vtk_file);

In [ ]:
const E = 2.1e11
const ν = 0.3

const λ = E*ν/(1+ν)/(1-2*ν)
const μ = E/2/(1+ν)




# function σ(eps)
#     # λ = E*ν/(1+ν)/(1-2*ν)
#     # μ = E/2/(1+ν)
#     # return lambda_ * ufl.nabla_div(u) * ufl.Identity(len(u)) + 2 * mu * epsilon(u)
#     return λ*tr(eps)*I(2) + 2*μ*eps
# end

σ(e) = λ*tr(e)*one(e) + 2*μ*e
# Model



In [ ]:

model = GmshDiscreteModel(name);

degree = 2
Ω = Triangulation(model)
dΩ = Measure(Ω,degree)

reffe = ReferenceFE(lagrangian,VectorValue{2,Float64},1)
V = TestFESpace(model,reffe,conformity=:H1,dirichlet_tags = ["left", "right"])

g0 = VectorValue(0.0,  0.0)
g1 = VectorValue(1.0mm, 0.0)

U = TrialFESpace(V, [g0,g1])


In [ ]:
# weak form 

# f = VectorValue(0.0, - gravity * ρ)
A(u,v) = ∫( ε(v) ⊙ (σ∘ε(u)) )*dΩ
L(v) = 0; #∫(dot(f, v))*dΩ

op = AffineFEOperator(A, L, U,V)

x0 = zeros(Float64, num_free_dofs(V))
uh_lin = FEFunction(U,x0)
ls = BackslashSolver()
solver = LinearFESolver(ls)

uh_lin, _ = solve!(uh_lin,solver,op)

In [ ]:
res_file = "results_new" |> tcf;

writevtk(Ω, res_file ,  cellfields=["uh"=>uh_lin,"sigma"=>σ∘ε(uh_lin)])